<a href="https://colab.research.google.com/github/geocarvalho/python-ds/blob/master/alura/ml_validacao_de_modelos/ml_validacao_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validação de modelos

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

X = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

SEED = 158020
np.random.seed(SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7500, 3), (2500, 3), (7500,), (2500,))

In [3]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(X_train, y_train)
acc = dummy_stratified.score(X_test, y_test) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acc)

A acurácia do dummy stratified foi 50.96%


In [4]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)

acc = accuracy_score(y_test, previsoes)*100
print("A acurácia foi %.2f%%" % acc)

A acurácia foi 71.92%


# Validação cruzada

* Treino/teste (holdout)
* Divide em k elementos deixando um de fora (leave one out)

In [5]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv=5, return_train_score=False)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print("Acurácia com validação cruzada com cv=5 é [%.2f; %.2f]" % ((media - 2*desvio_padrao)*100, (media + 2*desvio_padrao)*100))

Acurácia com validação cruzada com cv=5 é [75.21; 76.35]


* A validação cruzada não tem aleatoriedade, é determinístico
* Precisamos embaralhar e depois separar os *folds*

# Aleatoriedade na validação cruzada

In [0]:
def imprime_resultados(resultados):
  media = resultados['test_score'].mean()
  desvio_padrao = resultados['test_score'].std()
  print("Acurácia %.2f" % (media*100))
  print("Intervalo de acurácia com validação cruzada com cv=5: [%.2f; %.2f]" % ((media - 2*desvio_padrao)*100, (media + 2*desvio_padrao)*100))


In [7]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=5)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv=cv, return_train_score=False)
imprime_resultados(resultados)

Acurácia 75.78
Intervalo de acurácia com validação cruzada com cv=5: [75.33; 76.23]


In [8]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=5, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv=cv, return_train_score=False)
imprime_resultados(resultados)

Acurácia 75.76
Intervalo de acurácia com validação cruzada com cv=5: [73.46; 78.06]


# Simular situação de azar

* Pode ser azar ou uma proporção de exemplos desbalanceado entre as classes

In [9]:
dados_azar = dados.sort_values('vendido', ascending=True)
X_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']
dados_azar.head()


,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [10]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X_azar, y_azar, cv=cv, return_train_score=False)
imprime_resultados(resultados)

Acurácia 57.84
Intervalo de acurácia com validação cruzada com cv=5: [34.29; 81.39]


In [11]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X_azar, y_azar, cv=cv, return_train_score=False)
imprime_resultados(resultados)

Acurácia 75.78
Intervalo de acurácia com validação cruzada com cv=5: [72.30; 79.26]


In [12]:
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X_azar, y_azar, cv=cv, return_train_score=False)
imprime_resultados(resultados)

Acurácia 75.78
Intervalo de acurácia com validação cruzada com cv=5: [72.94; 78.62]


# Gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar nosso estimador

In [13]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=len(dados))
dados['modelo'] = dados.modelo + abs(dados.modelo.min()) + 1 # modelo inicial = 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [14]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [15]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

# Testando validação cruzada com GroupKFold

In [16]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(
    modelo, X_azar, y_azar, cv=cv, return_train_score=False, groups=dados['modelo'])
imprime_resultados(resultados)

Acurácia 75.78
Intervalo de acurácia com validação cruzada com cv=5: [73.67; 77.90]


# Validação cruzada com StandardScaler

In [19]:
SEED=301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(X_train, y_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

modelo = SVC()
modelo.fit(X_train_scaled, y_train)
previsoes = modelo.predict(X_test_scaled)

acc = accuracy_score(y_test, previsoes) * 100
print("A acurácia foi %.2f%%" % acc)

A acurácia foi 74.40%


In [20]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
resultados = cross_validate(
    modelo, X_azar, y_azar, cv=cv, return_train_score=False, groups=dados['modelo'])
imprime_resultados(resultados)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

Acurácia 58.00
Intervalo de acurácia com validação cruzada com cv=5: [56.10; 59.89]


In [23]:
scaler = StandardScaler()
scaler.fit(X_azar)
X_azar_escalado = scaler.transform(X_azar)

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
resultados = cross_validate(
    modelo, X_azar_escalado, y_azar, cv=cv, return_train_score=False, groups=dados['modelo'])
imprime_resultados(resultados)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

Acurácia 76.71
Intervalo de acurácia com validação cruzada com cv=5: [74.30; 79.12]


In [27]:
# Não se deve escalar antes da validação cruzada, mas sim para cada fase de treino

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()
pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
resultados = cross_validate(
    pipeline, X_azar, y_azar, cv=cv, return_train_score=False, groups=dados['modelo'])
imprime_resultados(resultados)

Acurácia 76.68
Intervalo de acurácia com validação cruzada com cv=5: [74.28; 79.08]
